In [1]:
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as dsets
import torchvision

In [2]:
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
training_path = "/content/drive/MyDrive/data/Training"
testing_path = "/content/drive/MyDrive/data/Testing"

In [5]:
for label, subdir in enumerate([d for d in os.listdir(training_path) if not d.startswith(".")]):
  print(label, subdir)

0 meningioma_tumor
1 no_tumor
2 pituitary_tumor
3 glioma_tumor


In [6]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [7]:
train_data = dsets.ImageFolder(training_path, train_transform)
test_data = dsets.ImageFolder(testing_path, test_transform)

In [8]:
batch_size = 32

train_loader = DataLoader(train_data,
                          batch_size=batch_size,
                          shuffle=True)

test_loader = DataLoader(test_data,
                         batch_size=batch_size,
                         shuffle=True)

In [9]:
num_classes = 4

model = models.inception_v3(pretrained=True)
model.aux_logits = False

for parameter in model.parameters():
    parameter.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 10),
    nn.Linear(10, num_classes)
)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 153MB/s] 


In [10]:
def train_model(model, train_loader, criterion, optimizer, start_epoch, num_epochs, resume_training=False):
    if resume_training:
        start_epoch, _ = load_checkpoint()

    for epoch in range(start_epoch, num_epochs):
        model.train()
        total_batch = len(train_loader)

        for i, (inputs, labels) in enumerate(train_loader):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % 5 == 0:
                print('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' % (epoch + 1, num_epochs, i + 1, total_batch, loss.item()))


        save_checkpoint({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss.item(),
        })

In [11]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [12]:
def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)

In [13]:
def load_checkpoint(filename='checkpoint.pth.tar'):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return epoch, loss

In [14]:
num_epochs = 10

In [18]:
param_grid = {
    'lr': [0.1, 0.01, 0.001],
    'batch_size': [16, 32, 64],
    'num_epochs': [10, 20]
}

grid = ParameterGrid(param_grid)
best_accuracy = 0
best_params = None

resume_training = True

for params in grid:
    lr = params['lr']
    batch_size = params['batch_size']
    num_epochs = params['num_epochs']

    print(f'Tuning for lr: {lr}, batch size: {batch_size}, epochs: {num_epochs}')

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)

    train_model(model, train_loader, criterion, optimizer, start_epoch=10, num_epochs=num_epochs, resume_training=resume_training)
    accuracy = evaluate_model(model, test_loader)

    print(f'Accuracy with lr: {lr}, batch size: {batch_size}, epochs: {num_epochs} -> {accuracy:.2f}%')

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = params

print(f'Best Hyperparameters: {best_params}')
print(f'Best Accuracy: {best_accuracy:.2f}%')

Tuning for lr: 0.1, batch size: 16, epochs: 10
Epoch [10/10], Iter [5/180] Loss: 7.1311
Epoch [10/10], Iter [10/180] Loss: 2.0340
Epoch [10/10], Iter [15/180] Loss: 15.8587
Epoch [10/10], Iter [20/180] Loss: 12.4928
Epoch [10/10], Iter [25/180] Loss: 7.8238
Epoch [10/10], Iter [30/180] Loss: 10.7009
Epoch [10/10], Iter [35/180] Loss: 6.0062
Epoch [10/10], Iter [40/180] Loss: 1.6998
Epoch [10/10], Iter [45/180] Loss: 18.8166
Epoch [10/10], Iter [50/180] Loss: 7.2508
Epoch [10/10], Iter [55/180] Loss: 9.4678
Epoch [10/10], Iter [60/180] Loss: 16.1193
Epoch [10/10], Iter [65/180] Loss: 9.4579
Epoch [10/10], Iter [70/180] Loss: 20.2921
Epoch [10/10], Iter [75/180] Loss: 20.8314
Epoch [10/10], Iter [80/180] Loss: 16.2091
Epoch [10/10], Iter [85/180] Loss: 6.8384
Epoch [10/10], Iter [90/180] Loss: 22.2102
Epoch [10/10], Iter [95/180] Loss: 7.8119
Epoch [10/10], Iter [100/180] Loss: 21.0368
Epoch [10/10], Iter [105/180] Loss: 8.5467
Epoch [10/10], Iter [110/180] Loss: 10.6333
Epoch [10/10], I

KeyboardInterrupt: 

In [19]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:

    images = images
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of test images: %f %%' % (100 * float(correct) / total))

Accuracy of test images: 49.238579 %
